# Migration between Scotland and overseas by age

Tabs: `AG 2001-02`, etc.

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/' \
                      'statistics-by-theme/migration/migration-statistics/migration-between-scotland-and-overseas')
#     scraper.run()
    distribution = scraper.distribution(
        mediaType='application/vnd.ms-excel',
        title='Migration between Scotland and overseas by age')
    display(distribution)
    tabs = [tab for tab in distribution.as_databaker() if tab.name.startswith('AG ')]
    display([tab.name for tab in tabs])
    

['AG 2001-02',
 'AG 2002-03',
 'AG 2003-04',
 'AG 2004-05',
 'AG 2005-06',
 'AG 2006-07',
 'AG 2007-08',
 'AG 2008-09',
 'AG 2009-10',
 'AG 2010-11',
 'AG 2011-12',
 'AG 2012-13',
 'AG 2013-14',
 'AG 2014-15',
 'AG 2015-16',
 'AG 2016-17']

In [2]:
Final_table = pd.DataFrame()

In [3]:
for tab in [t for t in tabs if t.name.startswith('AG ')]:
    flow = tab.filter('IN') | tab.filter('OUT') | tab.filter('NET')
    percentage = tab.filter('%').fill(DOWN).is_not_blank().is_not_whitespace()
    exclude = tab.filter(contains_string('Total net migration')).fill(DOWN).fill(RIGHT)
    observations = flow.fill(RIGHT).is_not_blank().is_not_whitespace() - percentage -exclude
    area = tab.filter(contains_string('Movements')).is_not_number()
    age = area.shift(0,1).expand(RIGHT).is_not_blank()
    Dimensions = [
            HDimConst('Mid Year',str(tab.name[3:7])+'-06-30T00:00:00/P1Y'),
            HDim(flow,'Flow',DIRECTLY, LEFT),
            HDim(age,'Age',DIRECTLY, ABOVE),
#             HDim(per,'per',CLOSEST, LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','T'),
            HDim(area,'Foreign geography',CLOSEST,ABOVE)
            ]
    c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)    
    new_table = c1.topandas()
    new_table['Flow'] = new_table['Flow'].map(
    lambda x: {
        'IN' : 'Inflow', 
        'OUT' : 'Outflow',
        'NET': 'Balance' 
        }.get(x, x))
    new_table['Age'] = new_table['Age'].map(
    lambda x: {
        '85+' : '85-plus', 
        'All ages' : 'all'        
        }.get(x, x))
    new_table['Age'] = 'nrs/' + new_table['Age']
    new_table['Mid Year'] = new_table['Mid Year'].map(lambda x: str(x)[0:4]) + '-06-30T00:00:00/P1Y'
    new_table['Domestic geography'] = 'Scotland'
    new_table = new_table[['Domestic geography','Foreign geography','Mid Year','Sex','Age','Flow','Measure Type','OBS','Unit']]
    Final_table = pd.concat([Final_table, new_table])

In [4]:
import numpy as np
Final_table['OBS'].replace('', np.nan, inplace=True)
Final_table.dropna(subset=['OBS'], inplace=True)
if 'DATAMARKER' in Final_table.columns:
    Final_table.drop(columns=['DATAMARKER'], inplace=True)
Final_table.rename(columns={'OBS': 'Value'}, inplace=True)
Final_table['Value'] = Final_table['Value'].astype(int)

In [5]:
Final_table['Foreign geography'] = Final_table['Foreign geography'].map(
    lambda x: {
        'Movements between Scotland and the rest of the UK1' : 'nrs/rest-of-the-uk', 
        'Movements between Scotland and overseas (including asylum seekers; excluding unmeasured migration adjustment)2, 3' : 'nrs/overseas',
        'Total net migration (including asylum seekers)3' : 'nrs/total',
        'Movements between Scotland and overseas (including asylum seekers; excluding unmeasured migration adjustment)2,3': 'nrs/overseas',
        'Movements between Scotland and Overseas (including asylum seekers)2':'nrs/overseas',
        'Total net migration (including asylum seekers)' : 'nrs/total'
        }.get(x, x))

In [6]:
Final_table

,Domestic geography,Foreign geography,Mid Year,Sex,Age,Flow,Measure Type,Value,Unit
0,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/0-15,Inflow,Count,9311,People
1,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/16-24,Inflow,Count,12787,People
2,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/25-34,Inflow,Count,13713,People
3,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/35-44,Inflow,Count,8038,People
4,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/45-54,Inflow,Count,4468,People
5,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/55-64,Inflow,Count,3257,People
6,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/65-74,Inflow,Count,1569,People
7,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/75-84,Inflow,Count,882,People
8,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/85+,Inflow,Count,372,People
9,Scotland,nrs/rest-of-the-uk,2001-06-30T00:00:00/P1Y,T,nrs/all,Inflow,Count,54397,People
